# Graveyard Of Wikidata - Statistics

Github : https://github.com/haikelJebahi/Graveyard-of-wikidata

Data : https://iccl.inf.tu-dresden.de/web/Wikidata_SPARQL_Logs/en

## Imports

In [ ]:
%matplotlib inline
from joblib import dump, load
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from urllib.parse import unquote_plus
from pyparsing import ParseException
from rdflib.plugins.sparql.algebra import translateQuery, translateUpdate, pprintAlgebra, traverse
from rdflib.plugins.sparql.parser import parseQuery, parseUpdate
from rdflib.plugins.sparql.operators import Builtin_LANG
from rdflib.plugins.sparql.parserutils import CompValue
from rdflib.term import Variable
from pandas import concat
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits import mplot3d
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import colorcet as cc
from sklearn.cluster import AgglomerativeClustering
from PIL import Image
import io
import gc
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
plt.style.use("ggplot")

In [ ]:
pd.set_option("display.max_rows", 0)
pd.set_option("display.max_columns", 0)
pd.set_option("display.expand_frame_repr", True)
pd.set_option("display.max_colwidth", -1)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


## Statistics

In [ ]:
nom = "../data/merged/status500_Joined.parsed.tsv"
df_nona = pd.read_csv(nom, sep="\t")


Adding basic attributes of queries

In [ ]:
df_nona["has_variable"] = df_nona["var_cpt"].apply(lambda x: False if x == 0 else True)
df_nona["has_filter"] = df_nona["filter"].apply(lambda x: False if x == 0 else True)
df_nona["has_orderby"] = df_nona["orderby"].apply(lambda x: False if x == 0 else True)
df_nona["has_limit"] = df_nona["limit"].apply(lambda x: False if x == 0 else True)
df_nona["has_select"] = df_nona["select"].apply(lambda x: False if x == 0 else True)
df_nona["has_distinct"] = df_nona["distinct"].apply(lambda x: False if x == 0 else True)
df_nona["has_modify"] = df_nona["modify"].apply(lambda x: False if x == 0 else True)
df_nona["has_tomultiset"] = df_nona["tomultiset"].apply(
    lambda x: False if x == 0 else True
)
df_nona["has_join"] = df_nona["join"].apply(lambda x: False if x == 0 else True)
df_nona["has_bgp"] = df_nona["bgp"].apply(lambda x: False if x == 0 else True)
df_nona["has_project"] = df_nona["project"].apply(lambda x: False if x == 0 else True)
df_nona["has_union"] = df_nona["union"].apply(lambda x: False if x == 0 else True)
df_nona["has_values"] = df_nona["values"].apply(lambda x: False if x == 0 else True)
df_nona["has_slice"] = df_nona["slice"].apply(lambda x: False if x == 0 else True)
df_nona["has_offset"] = df_nona["offset"].apply(lambda x: False if x == 0 else True)

df_nona["complexPathWith"] = df_nona.loc[:,['pathWithStar','pathWithQuestionMark','pathWithPlus']].sum(axis=1)
df_nona["simplePathWith"] = df_nona.loc[:,['pathWithInv','pathWithSequence','pathWithAlternative']].sum(axis=1)

df_nona

Split in robotic and organic

In [ ]:
df_nona_robotic = df_nona.loc[(df_nona["sourceCategory"] == "robotic")]
df_nona_robotic


,anonymizedQuery,timestamp,sourceCategory,user_agent,var_cpt,filter,orderby,limit,select,distinct,join,project,tomultiset,union,modify,bgp,values,groupBy,slice,triples,offset,extend,filter_not_exists,leftJoin,minus,notexist,sample,count,having,pathWithStar,pathWithPlus,pathWithQuestionMark,pathWithInv,pathWithSequence,pathWithAlternative,complexPathWith,simplePathWith,file_path,interval,has_variable,has_filter,has_orderby,has_limit,has_select,has_distinct,has_modify,has_tomultiset,has_join,has_bgp,has_project,has_union,has_values,has_slice,has_offset
0,SELECT+%3Fvar1++%3Fvar1Label++%3Fvar2+%28++CONCAT+%28++%22string1%22%2C+REPLACE+%28++STR+%28++%3Fvar3++%29+%2C+%22string2%22%2C+%22%22+%29+%2C+%22%5D%5D%22+%29++AS++%3Fvar4++%29+%3Fvar2Label++%3Fvar5+%0AWHERE+%7B%0A++%3Chttp%3A%2F%2Fwww.bigdata.com%2FqueryHints%23Query%3E++%3Chttp%3A%2F%2Fwww.bigdata.com%2FqueryHints%23optimizer%3E++%22None%22.%0A++%3Fvar2++%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23propertyType%3E++%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23Monolingualtext%3E+.%0A++%3Fvar2++%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23claim%3E++%3Fvar3+.%0A++%3Fvar1++%3Fvar3++%3Fvar6+.%0A++%3Fvar2++%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23statementProperty%3E++%3Fvar7+.%0A++%3Fvar6++%3Fvar7++%3Fvar5+.%0A+FILTER+%28++%28+%28++LANG+%28++%3Fvar5++%29++%3D++%22und%22+%29+%29+%0A%29+.%0A+FILTER+%28++%28+%28++%3Fvar1++%21%3D++%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2FQ22282914%3E++%29+%29+%0A%29+.%0A+SERVICE++%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23label%3E+++%7B%0A++++%3Chttp%3A%2F%2Fwww.bigdata.com%2Frdf%23serviceParam%3E++%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23language%3E++%22en%22.%0A++%7D%0A%7D%0AORDER+BY++DESC%28+%3Fvar2Label+%29ASC%28+%3Fvar5+%29%0ALIMIT+5000%0A,2017-06-12 00:16:34,robotic,bot2,9,1,1,5000,1,0,6,1,0,0,0,1,0,0,1,6,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,data/parsed/I1_status500_Joined.parsed.tsv,1,True,True,True,True,True,False,False,False,True,True,True,False,False,True,False
6,SELECT+DISTINCT+%3Fvar1++%3Fvar2+%0AWHERE+%7B%0A++BIND+%28++%28++%28++YEAR+%28++%3Fvar3++%29+-+YEAR+%28++%3Fvar4++%29++%29+-+IF+%28+%28++MONTH+%28++%3Fvar3++%29++%3C++MONTH+%28++%3Fvar4++%29++%29+%7C%7C+%28++MONTH+%28++%3Fvar3++%29++%3D++MONTH+%28++%3Fvar4++%29++%29+%26%26+%28++DAY+%28++%3Fvar3++%29++%3C++DAY+%28++%3Fvar4++%29++%29%2C%221%22%5E%5E%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23integer%3E+%2C%220%22%5E%5E%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23integer%3E++%29++%29++AS++%3Fvar2+%29.%0A++%3Fvar1++%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2FP31%3E++%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2FQ5%3E+%3B%0A+%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2FP569%3E++%3Fvar4+%3B%0A+%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2FP570%3E++%3Fvar3+.%0A++%3Fvar5++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+%3B%0A+%3Chttp%3A%2F%2Fschema.org%2FisPartOf%3E++%3Chttps%3A%2F%2Fsv.wikipedia.org%2F%3E+.%0A++%3Fvar1+%28+%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2FP569%3E+%2F+%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fstatement%2Fvalue%2FP569%3E+%29+%3Fvar6+.%0A++%3Fvar6++%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23timePrecision%3E++%2211%22%5E%5E%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23integer%3E+.%0A++%3Fvar1+%28+%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2FP570%3E+%2F+%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fstatement%2Fvalue%2FP570%3E+%29+%3Fvar7+.%0A++%3Fvar7++%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23timePrecision%3E++%2211%22%5E%5E%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23integer%3E+.%0A+FILTER+%28++%28+%28++%3Fvar2++%3E%3D+%22100%22%5E%5E%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23integer%3E++%29+%29+%0A%29+.%0A+FILTER+%28++%28+%28++YEAR+%28++%3Fvar4++%29++%3E+%221000%22%5E%5E%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23integer%3E++%29+%29+%0A%29+.%0A%7D%0AORDER+BY++DESC%28+%3Fvar2+%29%0A,2017-06-12 01:02:37,robotic,bot2,7,1,1,0,1,1,9,1,0,0,0,2,0,0,0,9,0,1,0,0,0,0,0,0,5,0,0,0,0,2,0,0,2,data/parsed/I1_status500_Joined.parsed.tsv,1,True,True,True,False,True,True,False,False,True

In [ ]:
df_nona_organic = df_nona.loc[(df_nona["sourceCategory"] == "organic")]
df_nona_organic


,anonymizedQuery,timestamp,sourceCategory,user_agent,var_cpt,filter,orderby,limit,select,distinct,join,project,tomultiset,union,modify,bgp,values,groupBy,slice,triples,offset,extend,filter_not_exists,leftJoin,minus,notexist,sample,count,having,pathWithStar,pathWithPlus,pathWithQuestionMark,pathWithInv,pathWithSequence,pathWithAlternative,complexPathWith,simplePathWith,file_path,interval,has_variable,has_filter,has_orderby,has_limit,has_select,has_distinct,has_modify,has_tomultiset,has_join,has_bgp,has_project,has_union,has_values,has_slice,has_offset
1,SELECT+%3Fvar1+%0AWHERE+%7B%0A++%3Fvar1++%3Chttp%3A%2F%2Fwww.w3.org%2F2004%2F02%2Fskos%2Fcore%23altLabel%3E++%3Fvar2+.%0A+FILTER+%28++%28++REGEX+%28++%3Fvar2+%2C+%22string1%22+%29++%29+%0A%29+.%0A%7D%0A,2017-06-12 00:27:10,organic,browser,2,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,data/parsed/I1_status500_Joined.parsed.tsv,1,True,True,False,False,True,False,False,False,False,True,True,False,False,False,False
2,SELECT+%3Fvar1+%0AWHERE+%7B%0A++%3Fvar1+%28+%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2FP279%3E+%7C+%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2FP31%3E+%29+%3Fvar2+.%0A++%3Fvar1++%3Chttp%3A%2F%2Fwww.w3.org%2F2004%2F02%2Fskos%2Fcore%23altLabel%3E++%3Fvar3+.%0A+FILTER+%28++%28++REGEX+%28++%3Fvar3+%2C+%22string1%22+%29++%29+%0A%29+.%0A%7D%0A,2017-06-12 00:33:31,organic,browser,3,1,0,0,1,0,1,1,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,data/parsed/I1_status500_Joined.parsed.tsv,1,True,True,False,False,True,False,False,False,True,True,True,False,False,False,False
3,SELECT%28+COUNT+%28*+%29+AS++%3Fvar1++%29+%3Fvar2++%3Fvar3Label++%3Fvar4++%3Fvar5Label++%3Fvar6++%3Fvar7Label++%3Fvar8++%3Fvar9Label+%0AWHERE+%7B%0A+SELECT%28++%3Fvar2++AS++%3Fvar10++%29+%3Fvar11++%3Fvar12Label++%3Fvar2++%3Fvar3Label++%3Fvar13++%3Fvar14Label++%3Fvar6++%3Fvar7Label++%3Fvar8++%3Fvar9Label+%0A+WHERE++%7B%0A+++BIND+%28++STRAFTER+%28++STR+%28++%3Fvar12++%29+%2C+%22%2Fentity%2F%22+%29++AS++%3Fvar11+%29.%0A+++BIND+%28++STRAFTER+%28++STR+%28++%3Fvar15++%29+%2C+%22%2Fdirect%2F%22+%29++AS++%3Fvar2+%29.%0A+++BIND+%28++STRAFTER+%28++STR+%28++%3Fvar14++%29+%2C+%22%2Fentity%2F%22+%29++AS++%3Fvar13+%29.%0A+++BIND+%28++STRAFTER+%28++STR+%28++%3Fvar7++%29+%2C+%22%2Fentity%2F%22+%29++AS++%3Fvar6+%29.%0A+++BIND+%28++STRAFTER+%28++STR+%28++%3Fvar9++%29+%2C+%22%2Fentity%2F%22+%29++AS++%3Fvar8+%29.%0A+++BIND+%28++STRAFTER+%28++STR+%28++%3Fvar16++%29+%2C+%22%2Fdirect%2F%22+%29++AS++%3Fvar4+%29.%0A+++%3Fvar12++%3Fvar15++%3Fvar14+.%0A+++%3Fvar12++%3Fvar16++%3Fvar7+.%0A+++%3Fvar14++%3Fvar16++%3Fvar9+.%0A+FILTER+%28+++%28+%28++%3Fvar15++%3D++%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2FP36%3E++%29+%29+%0A%29+.%0A+FILTER+%28+++%28+%28++%3Fvar16++%3D++%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2FP31%3E++%29+%29+%0A%29+.%0A+++%3Fvar3++%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23directClaim%3E++%3Fvar15+.%0A+++%3Fvar5++%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23directClaim%3E++%3Fvar16+.%0A++SERVICE++%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23label%3E++++%7B%0A+++++%3Chttp%3A%2F%2Fwww.bigdata.com%2Frdf%23serviceParam%3E++%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23language%3E++%22en%2Cfr%2Cde%2Cit%2Ces%22.%0A+++%7D%0A+%7D%0A%7D%0AGROUP+BY++%3Fvar2++%3Fvar3Label++%3Fvar4++%3Fvar5Label++%3Fvar6++%3Fvar7Label++%3Fvar8++%3Fvar9Label+%0A,2017-06-12 00:40:53,organic,browser,31,1,0,0,1,0,6,2,1,0,0,2,0,1,0,5,0,16,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,data/parsed/I1_status500_Joined.parsed.tsv,1,True,True,False,False,True,False,False,True,True,True,True,False,False,False,False
4,SELECT%28+COUNT+%28*+%29+AS++%3Fvar1++%29+%3Fvar2++%3Fvar3Label++%3Fvar4++%3Fvar5Label++%3Fvar6++%3Fvar7Label++%3Fvar8++%3Fvar9Label+%0AWHERE+%7B%0A+SELECT%28++%3Fvar2++AS++%3Fvar10++%29+%3Fvar11++%3Fvar12Label++%3Fvar2++%3Fvar3Label++%3Fvar13++%3Fvar14Label++%3Fvar6++%3Fvar7Label++%3Fvar8++%3Fvar9Label+%0A+WHERE++%7B%0A+++BIND+%28++%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2FP31%3E++AS++%3Fvar15+%29.%0A+++BIND+%28++STRAFTER+%28+

In [ ]:
len(df_nona)


122832

In [ ]:

data = dict()
for i in range(1,8):
    df_nona_interval = df_nona[df_nona["interval"] == i]
    for type in ["robotic", "organic"]:
        df_nona_interval_type = df_nona_interval[df_nona_interval["sourceCategory"] == type]
        
        sumVariable = len(df_nona_interval_type[df_nona_interval_type["has_variable"]==True])
        sumFilter = len(df_nona_interval_type[df_nona_interval_type["has_filter"]==True])
        sumOrderBy = len(df_nona_interval_type[df_nona_interval_type["has_orderby"]==True])
        sumLimit = len(df_nona_interval_type[df_nona_interval_type["has_limit"]==True])
        sumSelect = len(df_nona_interval_type[df_nona_interval_type["has_select"]==True])
        sumDistinct = len(df_nona_interval_type[df_nona_interval_type["has_distinct"]==True])
        sumToMultiSet = len(df_nona_interval_type[df_nona_interval_type["has_tomultiset"]==True])
        sumJoin = len(df_nona_interval_type[df_nona_interval_type["has_join"]==True])
        sumBGP = len(df_nona_interval_type[df_nona_interval_type["has_bgp"]==True])
        sumproject = len(df_nona_interval_type[df_nona_interval_type["has_project"]==True])
        sumUnion = len(df_nona_interval_type[df_nona_interval_type["has_union"]==True])
        sumValues = len(df_nona_interval_type[df_nona_interval_type["has_values"]==True])
        sumSlice = len(df_nona_interval_type[df_nona_interval_type["has_slice"]==True])
        sumSlice = len(df_nona_interval_type[df_nona_interval_type["has_offset"]==True])
        sumOffset = len(df_nona_interval_type[df_nona_interval_type["has_offset"]==True])

        data['I' + str(i) + " - " + type] = [sumVariable,sumValues,sumFilter,sumOrderBy,sumLimit,sumSelect,sumDistinct,sumSlice, sumToMultiSet,sumJoin,sumBGP,sumproject,sumUnion, sumOffset]

dpfr = pd.DataFrame(data=data, index=['Variable','Values','Filter','OrderBy','Limit','Select','Distinct','Slice', 'ToMultiSet','Join','BGP', 'project', 'Union',"Offset"])
dpfr

,I1 - robotic,I1 - organic,I2 - robotic,I2 - organic,I3 - robotic,I3 - organic,I4 - robotic,I4 - organic,I5 - robotic,I5 - organic,I6 - robotic,I6 - organic,I7 - robotic,I7 - organic
Variable,7272,3554,5372,5554,11101,5384,11097,5782,13153,6086,15839,6993,17522,8097
Values,380,90,253,152,426,294,1728,144,1718,155,2222,297,2932,265
Filter,4756,1376,4031,2711,6537,2630,8585,2269,11309,3174,12884,3477,13774,3896
OrderBy,1736,962,2545,1717,4461,1581,3043,1166,3398,1438,4182,1940,4139,2435
Limit,2271,1269,3584,2604,5851,2858,4850,3831,4674,3712,5454,4278,4552,4955
Select,7271,3543,5371,5551,11086,5375,11087,5759,13148,6069,15824,6972,17509,8033
Distinct,1083,843,673,1877,3798,1863,3132,1729,3790,2319,5356,2911,7444,3465
Slice,65,10,364,500,803,41,718,76,214,28,592,31,90,66
ToMultiSet,642,346,429,1046,534,1462,1849,1110,1987,1792,2691,2137,3594,2333
Join,6686,3217,4438,5112,9297,5035,10710,5364,12458,5727,14955,6527,15889,7604


### Top 10 with triples

In [ ]:
df_nona_sort_by_triples = df_nona.sort_values(["triples"],  ascending=[False])
df_nona_sort_by_triples_out = df_nona_sort_by_triples.head(10)
df_nona_sort_by_triples_out

,anonymizedQuery,timestamp,sourceCategory,user_agent,var_cpt,filter,orderby,limit,select,distinct,join,project,tomultiset,union,modify,bgp,values,groupBy,slice,triples,offset,extend,filter_not_exists,leftJoin,minus,notexist,sample,count,having,pathWithStar,pathWithPlus,pathWithQuestionMark,pathWithInv,pathWithSequence,pathWithAlternative,complexPathWith,simplePathWith,file_path,interval,has_variable,has_filter,has_orderby,has_limit,has_select,has_distinct,has_modify,has_tomultiset,has_join,has_bgp,has_project,has_union,has_values,has_slice,has_offset
53856,SELECT%28+COUNT+%28+DISTINCT++%3Fvar1++%29+AS++%3Fvar2++%29%0AWHERE+%7B%0A++%3Fvar3++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar3++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22en%22.%0A++%3Fvar4++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar4++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22de%22.%0A++%3Fvar5++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar5++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22fr%22.%0A++%3Fvar6++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar6++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22ja%22.%0A++%3Fvar7++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar7++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22es%22.%0A++%3Fvar8++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar8++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22ru%22.%0A++%3Fvar9++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar9++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22zh%22.%0A++%3Fvar10++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar10++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22it%22.%0A++%3Fvar11++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar11++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22pt%22.%0A++%3Fvar12++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar12++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22pl%22.%0A++%3Fvar13++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar13++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22nl%22.%0A++%3Fvar14++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar14++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22uk%22.%0A++%3Fvar15++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar15++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22fa%22.%0A++%3Fvar16++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar16++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22he%22.%0A++%3Fvar17++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar17++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22ar%22.%0A++%3Fvar18++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar18++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22sv%22.%0A++%3Fvar19++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar19++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22ko%22.%0A++%3Fvar20++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar20++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22cs%22.%0A++%3Fvar21++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar21++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22ca%22.%0A++%3Fvar22++%3Chttp%3A%2F%2Fschema.org%2Fabout%3E++%3Fvar1+.%0A++%3Fvar22++%3Chttp%3A%2F%2Fschema.org%2FinLanguage%3E++%22id%22.%0A%7D%0A,2017-12-29 05:57:47,organic,browser,23,0,0,0,1,0,39,1,0,0,0,1,0,1,0,40,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,data/parsed/I4_status500_Joined.parsed.tsv,4,True,False,False,False,True,False,False,False,True,True,True,False,False,False,False
101970,SELECT+*%0AWHERE+%7B%0A+%7B%0A+++%3Fvar1++%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2FP106%3E++%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2FQ177220%3E+.%0A+++%3Fvar1++%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2FP106%3E++%3Fvar2+.%0A+++%3Fvar1++%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2FP31%3E++%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2FQ5%3E+.%0A+++MINUS++++%7B%0A+++++%3Fvar1++%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2FP570%3E++%3Fvar3+.%0A+++%7D%0A+++%3Fvar1++%3Chttp%3A%2F

In [ ]:
%%script false --no-raise-error # disable cell

# Afficher la requête contenant le plus de triples
print(encoded2raw(
        df_nona_sort_by_triples_out.loc[53856]["anonymizedQuery"]
    ))

### Organic

Number of organic queries

In [ ]:
len(df_nona_organic)

41476

Sum of all variables

In [ ]:
df_nona_organic["var_cpt"].sum()

266963

Average number of variable per organic request

In [ ]:
df_nona_organic["var_cpt"].mean()

6.436565724756486

### Robotic

In [ ]:
len(df_nona_robotic)

81356

Sum of all variables

In [ ]:
df_nona_robotic["var_cpt"].sum()

448574

In [ ]:
df_nona_robotic["var_cpt"].mean()

5.513717488568759

## Graphs

In [ ]:
df_nona.sort_values(["cycleNumber"],  ascending=[False]).head(10)

In [ ]:
query = df_nona.sort_values(["cycleNumber"],  ascending=[False]).head(10).iloc[0]

triples = dict()
for triple in json.loads(query["triplesSet"]):
    triples[triple[0], triple[2]] = triple[1]

edges = triples
G = nx.DiGraph()
G.add_edges_from(edges)
pos = nx.spring_layout(G)
plt.figure()
nx.draw(
    G, pos,
    labels={node: node for node in G.nodes()},
)
#nx.draw_networkx_edge_labels(
#    G, pos,
#    edge_labels=triples,
#    font_color='red',
#    font_size = 8
#)
plt.axis('off')
plt.show()

print(list(nx.simple_cycles(G)))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=513ed372-d958-4618-b5dc-4b186f2555bd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>